In [1]:
import plotly.offline as plt
from plotly import tools
import plotly.graph_objs as go
from plotly.graph_objs import Layout
from plotly.graph_objs.graph_objs import Bar
from ipywidgets import *
import ipywidgets as widgets

plt.init_notebook_mode(connected=True)


In [7]:

class MultiHist(object):
    """Class for showing stacked histograms using plotly"""
    def __init__(self, titration):
        self.titration=titration
        self.x=self.titration[0]
        self.y=self.titration[1]
        self.slider=widgets.FloatSlider(max=5, step=0.001, orientation="horizontal")
        self.textfloat=BoundedFloatText(max=5, min=0, step=0.001)
        self.link=widgets.jslink((self.textfloat, 'value'), (self.slider, 'value'))
        self.stepSelector=widgets.Dropdown(options=['All']+[index+1 for index,step in enumerate(self.titration[0])],value='All',description='Step:',disabled=False,)

        
    def updateAttr(self, stepInt):
        xStep=[[chr(carac) for carac in range(1,100)]]*10
        yStep=[[0.01*num for num in range(1,100)], 
               [0.02*num for num in range(1,100)], 
               [0.03*num for num in range(1,100)], 
               [0.04*num for num in range(1,100)],
               [0.05*num for num in range(1,100)], 
               [0.06*num for num in range(1,100)],
               [0.07*num for num in range(1,100)], 
               [0.08*num for num in range(1,100)],
               [0.09*num for num in range(1,100)], 
               [0.1*num for num in range(1,100)]]
        if stepInt == 'All':
            self.x=self.titration[0]
            self.y=self.titration[1]
        else:
            self.x=[self.titration[0][stepInt-1]]
            self.y=[self.titration[1][stepInt-1]]
       
        
    def updateHist(self,stepInt, slider, textbox, ):
        """This function updates the cutoff value when it is modified by 1 of the 2 types of widget"""
        self.updateAttr(stepInt=stepInt)
        fig = tools.make_subplots(rows=len(self.y), cols=1, print_grid=False)
        fig['layout'].update(title='Titration step 1 to '+str(len(self.y)),shapes=[
                                    {
                                        'type': 'line',
                                        'xref': 'paper',
                                        'x0':-1,
                                        'y0': slider, # use absolute value or variable here
                                        'x1':len(self.y[0]),
                                        'y1': slider, # ditto
                                        'xref':"x",
                                        'yref':"y",
                                        'line': {
                                            'color': 'rgb(255,0,0)',
                                            'width': 1,
                                            'dash':'dashdot',
                                        },
                                    },
                                ]*len(self.y),showlegend=False,
                                    annotations=[{
                                            'x':-1,
                                            'y':2,
                                            'xref':'x',
                                            'yref':'y',
                                            'text':'max=5',
                                            'showarrow':False,
                                            'font':{
                                                'family':'Courier New, monospace',
                                                'size':11,
                                                'color':'#ffffff'
                                            },
                                            
                                            'bordercolor':'rgb(255,255,255)',
                                            'borderwidth':2,
                                            'borderpad':4,
                                            'bgcolor':'rgb(0,0,0)',
                                            'opacity':0.7
                                    }
                                    ]*len(self.y)+[{
                                            'xref': 'paper',
                                            'yref': 'paper',
                                            'x': 0,
                                            'xanchor': 'right',
                                            'y': 1,
                                            'yanchor': 'bottom',
                                            'text': 'Chemical shift intensity',
                                            'showarrow': False,
                                            'font':{
                                                'size':16,                                                
                                            },
                                          }, {
                                            'xref': 'paper',
                                            'yref': 'paper',
                                            'x': 1,
                                            'xanchor': 'left',
                                            'y': 0,
                                            'yanchor': 'top',
                                            'text': 'Amino acid number',
                                            'showarrow': False,
                                            'font':{
                                                'size':16,                                                
                                            }
                                          }],
                                    width=1000,
                                    height=1000,
                                    margin=go.Margin(
                                        l=200,
                                        r=200,
                                        b=100,
                                        t=100,
                                        pad=4
    )
                )
        
        for stepIndex,step in enumerate(self.y):
            color=[]
            for index,elm in enumerate(step):
                if elm > slider:
                    color.append('rgb(255,128,0)')
                else:
                    color.append('rgb(0,128,255)')
            trace1 = go.Bar(
                    x=self.x[stepIndex],
                    y=self.y[stepIndex],
                    marker=dict(
                        color=color),
                    name='Step '+str(stepIndex+1),
                )
            fig.append_trace(trace1, stepIndex+1, 1)
            fig['layout']['shapes'][stepIndex]['xref']="x"+str(stepIndex+1)
            fig['layout']['shapes'][stepIndex]['yref']="y"+str(stepIndex+1)
            fig['layout']['annotations'][stepIndex]['xref']="x"+str(stepIndex+1)
            fig['layout']['annotations'][stepIndex]['yref']="y"+str(stepIndex+1)
            fig['layout']['annotations'][stepIndex]['x']=(len(self.x[stepIndex]))/2
            fig['layout']['annotations'][stepIndex]['y']=1.5*max(self.y[-1])
            fig['layout']['annotations'][stepIndex]['text']="Step " +str(stepIndex+1)
            fig['layout']['yaxis'+str(stepIndex+1)].update(range=[0,1.8*max(self.y[-1])])
        if stepInt != 'All':
            fig['layout'].update(xaxis={'showline':True, 'showgrid':True,'zeroline':True}, yaxis={'range':[0,max(self.y[stepIndex])],'showline':True,'zeroline':False})
            fig['layout']['annotations']=[]
            fig['layout']['title']="Titration step "+str(stepInt)
        plt.iplot(fig)

titrationExample=[[[chr(carac) for carac in range(1,100)]]*10,
[[0.01*num for num in range(1,100)],
  [0.02*num for num in range(1,100)], 
  [0.03*num for num in range(1,100)], 
  [0.04*num for num in range(1,100)],
  [0.05*num for num in range(1,100)], 
  [0.06*num for num in range(1,100)],
  [0.07*num for num in range(1,100)], 
  [0.08*num for num in range(1,100)],
  [0.09*num for num in range(1,100)], 
  [0.1*num for num in range(1,100)]]]
h=MultiHist(titrationExample)


In [8]:
widgets.interactive(h.updateHist, stepInt=h.stepSelector, slider=h.slider, textbox=h.textfloat, )

A Jupyter Widget